In [ ]:
import math
import weakref

In [ ]:
import carla

In [ ]:
NOISE_STDDEV = 5e-5
NOISE_BIAS = 1e-5

In [ ]:
class GnssSensor:
    def __init__(
        self,
        parent_actor,
        sensor_tick=0.1,
        noise_alt_bias=None,
        noise_alt_stddev=None,
        noise_lat_bias=None,
        noise_lat_stddev=None,
        noise_lon_bias=None,
        noise_lon_stddev=None
    ):
        self.sensor = None
        self._parent = parent_actor
        self.lat = 0.0
        self.lon = 0.0
        world = self._parent.get_world()
        bp = world.get_blueprint_library().find("sensor.other.gnss")
        if sensor_tick: # 传感器捕获频率
            bp.set_attribute("sensor_tick", sensor_tick)
        if noise_alt_bias:  # 海拔噪声模型中的均值
            bp.set_attribute("noise_alt_bias", noise_alt_bias)
        if noise_alt_stddev:    # 海拔噪声模型中的标准偏差
            bp.set_attribute("noise_alt_stddev", noise_alt_stddev)
        if noise_lat_bias:  # 纬度噪声模型中的均值
            bp.set_attribute("noise_lat_bias", noise_lat_bias)
        if noise_lat_stddev:    # 纬度噪声模型中的标准偏差
            bp.set_attribute("noise_lat_stddev", noise_lat_stddev)
        if noise_lon_bias:  # 经度噪声模型中的均值
            bp.set_attribute("noise_lon_bias", noise_lon_bias)
        if noise_lon_stddev:    # 经度噪声模型中的标准偏差
            bp.set_attribute("noise_lon_stddev", noise_lon_stddev)
        self.sensor = world.spawn_actor(blueprint=bp, transform=carla.Transform(carla.Location(x=1.0, z=2.8)), attach_to=self._parent)
        weak_self = weakref.ref(self)
        self.sensor.listen(lambda event: GnssSensor._gnss_callback(weak_self, event))

    @staticmethod
    def _gnss_callback(weak_self, event):
        self = weak_self()
        if not self:
            return
        self.lat = event.latitude
        self.lon = event.longitude

In [ ]:
class IMUSensor:
    def __init__(
        self,
        parent_actor,
        sensor_tick=0.01,
        noise_accel_stddev_x=None,
        noise_accel_stddev_y=None,
        noise_accel_stddev_z=None,
        noise_gyro_bias_x=None,
        noise_gyro_bias_y=None,
        noise_gyro_bias_z=None,
        noise_gyro_stddev_x=None,
        noise_gyro_stddev_y=None,
        noise_gyro_stddev_z=None
    ):
        self.sensor = None
        self._parent = parent_actor
        self.accelerometer = (0.0, 0.0, 0.0)
        self.gyroscope = (0.0, 0.0, 0.0)
        self.compass = 0.0
        world = self._parent.get_world()
        bp = world.get_blueprint_library().find("sensor.other.imu")
        if sensor_tick: # 传感器捕获频率
            bp.set_attribute("sensor_tick", sensor_tick)
        if noise_accel_stddev_x:    # 加速度噪声模型中的标准差（X轴）
            bp.set_attribute("noise_accel_stddev_x", noise_accel_stddev_x)
        if noise_accel_stddev_y:    # 加速度噪声模型中的标准差（Y轴）
            bp.set_attribute("noise_accel_stddev_y", noise_accel_stddev_y)
        if noise_accel_stddev_z:    # 加速度噪声模型中的标准差（Z轴）
            bp.set_attribute("noise_accel_stddev_z", noise_accel_stddev_z)
        if noise_gyro_bias_x:   # 陀螺仪噪声模型中的均值（X轴）
            bp.set_attribute("noise_gyro_bias_x", noise_gyro_bias_x)
        if noise_gyro_bias_y:   # 陀螺仪噪声模型中的均值（Y轴）
            bp.set_attribute("noise_gyro_bias_y", noise_gyro_bias_y)
        if noise_gyro_bias_z:   # 陀螺仪噪声模型中的均值（Z轴）
            bp.set_attribute("noise_gyro_bias_z", noise_gyro_bias_z)
        if noise_gyro_stddev_x: # 陀螺仪噪声模型中的标准差（X轴）
            bp.set_attribute("noise_gyro_stddev_x", noise_gyro_stddev_x)
        if noise_gyro_stddev_y: # 陀螺仪噪声模型中的标准差（Y轴）
            bp.set_attribute("noise_gyro_stddev_y", noise_gyro_stddev_y)
        if noise_gyro_stddev_z: # 陀螺仪噪声模型中的标准差（Z轴）
            bp.set_attribute("noise_gyro_stddev_z", noise_gyro_stddev_z)
        self.sensor = world.spawn_actor(blueprint=bp, transform=carla.Transform(), attach_to=self._parent)
        weak_self = weakref.ref(self)
        self.sensor.listen(lambda sensor_data: IMUSensor._imu_callback(weak_self, sensor_data))

    @staticmethod
    def _imu_callback(weak_self, sensor_data):
        self = weak_self()
        if not self:
            return
        limits = (-99.9, 99.9)
        self.accelerometer = (
            max(limits[0], min(limits[1], sensor_data.accelerometer.x)),
            max(limits[0], min(limits[1], sensor_data.accelerometer.y)),
            max(limits[0], min(limits[1], sensor_data.accelerometer.z)))
        self.gyroscope = (
            max(limits[0], min(limits[1], math.degrees(sensor_data.gyroscope.x))),
            max(limits[0], min(limits[1], math.degrees(sensor_data.gyroscope.y))),
            max(limits[0], min(limits[1], math.degrees(sensor_data.gyroscope.z))))
        self.compass = math.degrees(sensor_data.compass)

In [ ]:
class Vehicle:
    def __init__(self, world, client, spawn_point):
        self.world = world
        self.client = client